# Imports (Frequent Itemsets)


In [7]:
import sqlite3
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [8]:
db_connection = sqlite3.connect("go_sales.sqlite")
sql_query = "SELECT * FROM PRODUCT;"
lol = pd.read_sql_query(sql_query, db_connection)
sql_query = "SELECT * FROM ORDER_DETAILS;"
lol2 = pd.read_sql_query(sql_query, db_connection)



In [9]:
merged = pd.merge(lol, lol2, on="PRODUCT_NUMBER", how="left")

In [10]:
merged_skimmed = merged.loc[:, ["ORDER_NUMBER", "PRODUCT_NAME", "QUANTITY"]]
merged_skimmed

,ORDER_NUMBER,PRODUCT_NAME,QUANTITY
0,8474,TrailChef Water Bag,40
1,8470,TrailChef Water Bag,40
2,8464,TrailChef Water Bag,48
3,9254,TrailChef Water Bag,66
4,8472,TrailChef Water Bag,88
...,...,...,...
43058,8210,Aloe Relief,30
43059,8246,Aloe Relief,20
43060,8239,Aloe Relief,18
43061,8270,Aloe Relief,26


In [11]:
transactions = []

for order_num in merged_skimmed["ORDER_NUMBER"].unique():
    products = merged_skimmed[merged_skimmed["ORDER_NUMBER"] == order_num]["PRODUCT_NAME"].tolist()
    transactions.append(products)

te = TransactionEncoder()
df_dummies = te.fit_transform(transactions)
df = pd.DataFrame(df_dummies, columns=te.columns_)

print(df.head())

   Aloe Relief  Bear Edge  Bear Survival Edge  Blue Steel Max Putter  \
0        False      False               False                  False   
1        False      False               False                  False   
2        False      False               False                  False   
3        False      False               False                  False   
4        False      False               False                  False   

   Blue Steel Putter  BugShield Extreme  BugShield Lotion  \
0              False               True             False   
1              False              False             False   
2              False               True             False   
3              False              False             False   
4              False              False              True   

   BugShield Lotion Lite  BugShield Natural  BugShield Spray  ...  \
0                  False              False            False  ...   
1                  False              False            False  

In [12]:
frq_items1 = apriori(df, min_support=0.01, use_colnames=True)

combinations = frq_items1[frq_items1['itemsets'].apply(lambda x: len(x)) > 1]

print(combinations)

       support                                           itemsets
115   0.011381                   (BugShield Extreme, Aloe Relief)
116   0.010075                         (Double Edge, Aloe Relief)
117   0.013993                         (Aloe Relief, Sun Blocker)
118   0.011194                      (Aloe Relief, Sun Shelter 15)
119   0.015858                      (Aloe Relief, Sun Shelter 30)
...        ...                                                ...
1902  0.016978       (TrailChef Water Bag, TrailChef Kitchen Kit)
1903  0.014552       (TrailChef Single Flame, TrailChef Utensils)
1904  0.012313      (TrailChef Single Flame, TrailChef Water Bag)
1905  0.015672          (TrailChef Water Bag, TrailChef Utensils)
1906  0.010448  (Lady Hailstorm Titanium Irons, Blue Steel Max...

[1792 rows x 2 columns]
